In [10]:
import numpy as np
import pandas as pd
import openai
import time

In [2]:
openai.api_key = "your_api_key_here"

In [3]:
timc_data = pd.read_csv('./timc_recist_labeled_dataset_id.csv')

In [ ]:
training_timc = timc_data[timc_data.split=='train'][['obs','date','overall_response']].groupby(['obs','date']).first().reset_index()
training_timc = training_timc.sort_values(by=['obs','date'])
training_timc['timepoint'] = training_timc.groupby('obs').cumcount()
training_timc['annotation'] = training_timc.timepoint.astype(str) + ". " + training_timc.overall_response
training_timc.head()

In [5]:
temp = training_timc[training_timc.obs==0]

In [6]:
temp2 = " \n ".join(temp.annotation.tolist())

In [7]:
base_prompt = """
Imagine a patient with advanced cancer; please choose the cancer type. The patient is treated on a clinical trial of a new drug. I will provide a list of timepoint-specific RECIST annotations corresponding to one or more imaging studies performed at that timepoint. For each timepoint annotation in the list, generate a list of one or more reports of a CT scan, PET/CT, bone scan, and/or MRI scan that could have been the basis for the RECIST annotation at that timepoint. Format your output as JSON, consisting of a list of timepoints, in which each timepoint element contains three fields. The first field should be the timepoint number ("timepoint_number"). The second should be the RECIST annotation ("recist_annotation"). The third should be a list of synthetic reports ("synthetic_reports") for that timepoint. The reports should be full-length radiology reports formatted as one string, including type of scan, narrative describing findings, and an impression. There should usually be multiple imaging studies performed at the each timepoint.
Here is an example of a report:
Examination: Computed Tomography (CT) of the Chest
Indication: Non-small cell lung cancer, restaging.
Technique: Axial images were obtained through the chest after the administration of intravenous contrast. Coronal and sagittal reformatted images were also reviewed.
Findings:
The primary tumor located in the right upper lobe appears to have decreased in size compared to the previous scan, measuring approximately 3.2 cm in the greatest dimension, previously 4.5 cm.
There is no evidence of new pulmonary nodules or masses. The previously noted mediastinal and hilar lymphadenopathy has also decreased in size.
There are no pleural or pericardial effusions. The heart size is within normal limits. The visualized upper abdomen is unremarkable.
Impression:
1. Decrease in size of the primary right upper lobe non-small cell lung cancer and associated lymphadenopathy, suggestive of response to therapy.
2. No new pulmonary nodules or masses.
3. No evidence of metastatic disease in the visualized upper abdomen.
(end example)

The list of annotations is as follows. Note that BL means baseline, PR means partial response, CR means complete response, SD means stable disease, and PD means progressive disease. \n"""


In [8]:
training_timc.obs.nunique()

5724

In [ ]:
for i in range(training_timc.obs.nunique()):
    thisobs = training_timc[training_timc.obs == i]
    annotation_string = " \n ".join(thisobs.annotation.tolist())
    prompt = base_prompt + annotation_string
    try:
        response = openai.ChatCompletion.create(
        model = "gpt-4-1106-preview",
        messages = [{"role": "user", "content": prompt}],
        temperature = 0.7,
        response_format = {'type': "json_object"}            
        )
    except:
        time.sleep(10)
        response = openai.ChatCompletion.create(
        model = "gpt-4-1106-preview",
        messages = [{"role": "user", "content": prompt}],
        temperature = 0.7,
        response_format = {'type': "json_object"}
        )
    
    output_text = response['choices'][0]['message']['content']
    with open('./synthetic_data/obs' + str(i) + '.txt', 'w') as f:
        f.write(output_text)
    
    if i % 10 == 0:
        print(i)
    
    
               